# Predicting Telecom Prediction 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.preprocessing import StandardScaler,RobustScaler,QuantileTransformer
from sklearn.feature_selection import SelectKBest,mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [3]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [4]:
df = pd.read_csv("/content/drive/MyDrive/DSC 550/data.csv")
df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_Male,Partner_Yes,Dependents_Yes,PhoneService,NonMult_PhoneService,...,NoStreamTV,StreamTV,NoStreamMovie,StreamingMovies,OneYrContract,TwoYrContract,PaperLessBilling,CreditCardPayment,ElectCheckPayment,MailedChkPayment
0,0,1,29.85,29.85,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,1,0
1,0,34,56.95,1889.50,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
2,0,2,53.85,108.15,1,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
3,0,45,42.30,1840.75,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,0,2,70.70,151.65,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0


#  Check for class imbalance and correct it

In [5]:
df['Churn'].value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [6]:
X = df.drop('Churn',axis=1)
y = df['Churn']

In [7]:
undersample = RandomUnderSampler(sampling_strategy='majority')
X_over,y_over = undersample.fit_resample(X,y)

In [8]:
y_over.value_counts()

0    1869
1    1869
Name: Churn, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
# Get X_train, X_test, y_train and y_test

X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.20, random_state=42)

# Set up a pipeline and search space

Setting up a pipeline

In [11]:
pipe = Pipeline([('scaler',StandardScaler()),
                 ('selector',SelectKBest(mutual_info_classif,k=7)),
                 ('classifier',LogisticRegression())])

Define the search Space

In [12]:
search_space = [{'classifier':[LogisticRegression()],
                 'classifier__penalty':['l1','l2'],
                 'classifier__C':np.logspace(0,4,10)},
                {'classifier':[KNeighborsClassifier()],
                'classifier__n_neighbors':[3,5,8,11],
                 'classifier__weights':['uniform','distance']},
                {'classifier':[RandomForestClassifier()],
                 'classifier__n_estimators':[10,100,1000],
                 'classifier__max_features':[1,2,3]}]

# Train the model

In [13]:
# Run the gridsearch

model = GridSearchCV(pipe,search_space,cv=10,verbose=0)
model.fit(X_train,y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 370.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k=7,
                                                    score_func=<function mutual_info_classif at 0x7f7bc472de60>)),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'classifier': [LogisticRegression(C=1291.5496650148827)],
                          'classifier__C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                          'classifier__penalty': ['l1', 'l2']},
                         {'classifier': [KNeighborsClassifier()],
                          'classifier__n_neighbors': [3, 5, 8, 11],
                          'classifier__weights': ['uniform', 'distance']},
                         

Obtain predictions for train and test sets

In [14]:
best_estm = model.best_estimator_
print(best_estm)

Pipeline(steps=[('scaler', StandardScaler()),
                ('selector',
                 SelectKBest(k=7,
                             score_func=<function mutual_info_classif at 0x7f7bc472de60>)),
                ('classifier', LogisticRegression(C=1291.5496650148827))])


# Evaluate the Model

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
y_pred_train = best_estm.predict(X_train)
mse = mean_squared_error(y_pred_train,y_train)
print("MSE: %2f" %mse)

MSE: 0.235452


In [17]:
y_pred_test = best_estm.predict(X_test)
mse = mean_squared_error(y_pred_test,y_test)
print("MSE: %2f" %mse)

MSE: 0.259358


Print classification Report

In [18]:
from sklearn.metrics import confusion_matrix,classification_report

In [19]:
cm = confusion_matrix(y_test,y_pred_test)

In [20]:
print(cm)

[[257 122]
 [ 72 297]]


In [21]:
print(classification_report(y_test,y_pred_test))

              precision    recall  f1-score   support

           0       0.78      0.68      0.73       379
           1       0.71      0.80      0.75       369

    accuracy                           0.74       748
   macro avg       0.74      0.74      0.74       748
weighted avg       0.75      0.74      0.74       748



Having adjusted my class size using random undersampler, the classification of those that churn has improved from 55% to 75%.  I also incorporated feature selection into the pipeline model to reduce unnecessary noise.
